In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install sklearn
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [3]:
import argparse
import json
import os
import logging
from functools import partial
# import pytorch_lightning as pl
import torch
from tqdm import trange
from tqdm.notebook import tqdm
# from pytorch_lightning import Trainer
# from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
# from pytorch_lightning.loggers import TensorBoardLogger
from torch.nn import functional as F
from torch.nn.modules import CrossEntropyLoss
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler, TensorDataset, ConcatDataset, Subset)
from transformers import AdamW, get_linear_schedule_with_warmup, RobertaTokenizer
from transformers import RobertaConfig, RobertaModel, AutoTokenizer, AutoModel
from transformers import PYTORCH_PRETRAINED_BERT_CACHE, WEIGHTS_NAME, CONFIG_NAME
# from torchmetrics import Accuracy
from typing import List
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score
from scipy.stats import pearsonr, spearmanr

logger = logging.getLogger(__name__)

# Args

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [5]:
device

device(type='cuda')

In [6]:
class parser():
  def __init__(self):
    self.bert_path = "vinai/phobert-base-v2"
    self.data_dir= "/content/drive/MyDrive/pRoBERTa/data/converted_data_train_tocken.json"
    self.save_path="/content/drive/MyDrive/pRoBERTa/trained_module/rationale_selection/self_explain_model"
    self.checkpoint_path="/content/drive/MyDrive/pRoBERTa/trained_module/rationale_selection/self_explain_model"
    self.batch_size=10
    self.epochs=10
    self.kfold = 10
    self.do_train = True
    self.do_eval = True
    self.do_test = False
    self.scale_pos_neg = 10
    self.num_labels = 2
    self.lr=2e-5
    self.workers=0
    self.weight_decay=0.0
    self.adam_epsilon=1e-9
    self.warmup_steps=0
    self.max_length=256
    self.save_topk=2
    self.span_topk=2
    self.lamb=1.0
    self.task='snli'
    self.mode='train'
    self.gradient_accumulation_steps = 1
    self.local_rank  = -1
    self.fp16 = False
args = parser()

# Prepare data

In [47]:
import random
from torch.utils.data import Dataset
import json
import pandas as pd
class LoadDataFromJson(Dataset, ):
    def __init__(self, datasets):
        self.samples = []
        with open(datasets) as file:
            data = json.load(file)

        for id, item in data.items():
            sentences = item["context"]
            if item["evidence"] != None:
              for sentence in sentences:
                data_sample = {}
                evidence = item['evidence'] in sentence or sentence in item['evidence']
                if evidence:
                  data_sample['evidence_label'] = 1
                else:
                  data_sample['evidence_label'] = 0
                data_sample['id'] = id
                data_sample['claim'] = item['claim']
                data_sample['sentence'] = sentence
                data_sample['evidence'] = item['evidence']

                data_sample["verdict"] = item["verdict"]
                data_sample['domain'] = item['domain']
                self.samples.append(data_sample)

    def to_dataframe(self):
        return pd.DataFrame(self.samples).sample(frac=1, random_state=3)


In [48]:
dat_path = args.data_dir
a = LoadDataFromJson(dat_path)
df = a.to_dataframe()
df

,evidence_label,id,claim,sentence,evidence,verdict,domain
234309,0,30927,khi gia_nhập college compass nhung được kết_nố...,trong bài_luận gửi đến trường harvard nữ_sinh_...,khi trở_thành học_sinh của college compass nhu...,SUPPORTED,giaoduc
223745,0,30428,từ lúc nghỉ_việc gia_đình chị thúy rất khó khắ...,lê tuyết,từ lúc nghỉ_việc gia_đình chị thúy chỉ kiếm đư...,SUPPORTED,thoisu
94193,0,11558,cần nhiều công_nghệ và thời_gian để khắc_phục ...,khi tìm_kiếm sản_phẩm chăm_sóc da đặc_biệt cho...,bác_sĩ thành cho rằng trường_hợp bệnh_nhân trê...,SUPPORTED,suckhoe
114060,0,12370,ông trump không_thể tự ân_xá cho mình dù trở_t...,quá_trình xử_lý vụ án có_thể diễn ra trong thờ...,nếu trở_thành tổng_thống ông trump không nắm g...,SUPPORTED,thegioi
193742,0,19637,phòng lao_động quận 7 ban quản_lý các khu chế_...,khảo_sát của công_đoàn cho thấy khoảng 240 lao...,hơn tháng qua công_đoàn đã trực_tiếp liên_hệ v...,SUPPORTED,thoisu
...,...,...,...,...,...,...,...
11261,0,7631,tháng 4/2020 là thời_kỳ đỉnh_điểm nhất của cov...,đặc_biệt sản_phẩm này có tiềm_năng hỗ_trợ ngườ...,giai_đoạn cao_điểm covid19 hồi tháng 4 năm 202...,SUPPORTED,khoahoc
48056,0,9398,vật_liệu điện_sắc được giới khoa_học quan_tâm ...,thiết_bị điện_sắc được nhóm nghiên_cứu thử_ngh...,theo nhóm vật_liệu điện_sắc được giới khoa_học...,SUPPORTED,khoahoc
77049,0,10830,những chiến_thắng được ca_ngợi giúp prigozhin ...,các thành_viên wagner được cho là đã âm_thầm t...,điều đó cũng giúp prigozhin tăng_cường ảnh_hưở...,SUPPORTED,thegioi
198296,0,19911,những bức ảnh được nhiếp_ảnh gia_john thomson ...,áo_dài năm thân_may theo phong_cách kín_đáo đư...,thiếu_nữ sài_gòn là một trong những bức ảnh đư...,SUPPORTED,thegioi


In [49]:
df["evidence_label"].value_counts()


0    322111
1     14550
Name: evidence_label, dtype: int64

In [50]:
filtered_df = df.loc[(df['sentence'].apply(lambda x: len(x.split())) < 4)]
filtered_df

,evidence_label,id,claim,sentence,evidence,verdict,domain
223745,0,30428,từ lúc nghỉ_việc gia_đình chị thúy rất khó khắ...,lê tuyết,từ lúc nghỉ_việc gia_đình chị thúy chỉ kiếm đư...,SUPPORTED,thoisu
54650,0,9735,giảo cổ_lam,thanh hoa,giảo cổ_lam trong giảo cổ_lam có chứa phanosid,SUPPORTED,suckhoe
162702,0,14185,đầu năm 2022 dự_án sẽ đi vào hoạt_động,để phòng nguy_hiểm,đầu năm 2022 công_trình đi vào hoạt_động,SUPPORTED,thoisu
160332,0,14075,rạng sáng nay người_dân khu_vực gần cầu kênh n...,ngọc tài nguyễn_khánh,rạng sáng nay người_dân gần cầu bắc qua kênh n...,SUPPORTED,thoisu
214061,0,30071,trong năm đầu_tiên cô đã hoàn_thành_phần nghiê...,thanh hằng,năm đầu_tiên cô hoàn_thành_phần nghiên_cứu lý_...,SUPPORTED,giaoduc
...,...,...,...,...,...,...,...
227131,0,30588,trong thông_báo vừa gửi 7 hãng taxi về giá dịc...,gia_minh,trong thông_báo vừa gửi 7 hãng taxi về giá dịc...,SUPPORTED,thoisu
198769,0,19936,yoga xuất_hiện từ thời ấn_độ cổ_đại,bảo đức,yoga là bộ_môn truyền_thống năm nghìn năm_tuổi...,SUPPORTED,thegioi
185884,1,19243,dự_kiến tổng mức đầu_tư cho dự_án cảng_hàng_kh...,phước tuấn_gia chính,phước tuấn_gia chính,SUPPORTED,thoisu
306027,0,46364,ông nguyễn_quang_thái thành_viên hội_đồng tư_v...,hà_an,ông nguyễn_quang_thái thành_viên hội_đồng tư_v...,REFUTED,khoahoc


In [51]:
df = df.drop(df[df.isin(filtered_df.to_dict('list')).all(1)].index)
df


,evidence_label,id,claim,sentence,evidence,verdict,domain
234309,0,30927,khi gia_nhập college compass nhung được kết_nố...,trong bài_luận gửi đến trường harvard nữ_sinh_...,khi trở_thành học_sinh của college compass nhu...,SUPPORTED,giaoduc
94193,0,11558,cần nhiều công_nghệ và thời_gian để khắc_phục ...,khi tìm_kiếm sản_phẩm chăm_sóc da đặc_biệt cho...,bác_sĩ thành cho rằng trường_hợp bệnh_nhân trê...,SUPPORTED,suckhoe
114060,0,12370,ông trump không_thể tự ân_xá cho mình dù trở_t...,quá_trình xử_lý vụ án có_thể diễn ra trong thờ...,nếu trở_thành tổng_thống ông trump không nắm g...,SUPPORTED,thegioi
193742,0,19637,phòng lao_động quận 7 ban quản_lý các khu chế_...,khảo_sát của công_đoàn cho thấy khoảng 240 lao...,hơn tháng qua công_đoàn đã trực_tiếp liên_hệ v...,SUPPORTED,thoisu
349,0,7145,số_liệu của viện phát_triển giáo_dục hàn_quốc ...,tôi nghĩ không nhiều đứa trẻ gây ra nỗi đau_th...,lee cũng cho rằng nhiều học_sinh đã mất niềm t...,SUPPORTED,giaoduc
...,...,...,...,...,...,...,...
11261,0,7631,tháng 4/2020 là thời_kỳ đỉnh_điểm nhất của cov...,đặc_biệt sản_phẩm này có tiềm_năng hỗ_trợ ngườ...,giai_đoạn cao_điểm covid19 hồi tháng 4 năm 202...,SUPPORTED,khoahoc
48056,0,9398,vật_liệu điện_sắc được giới khoa_học quan_tâm ...,thiết_bị điện_sắc được nhóm nghiên_cứu thử_ngh...,theo nhóm vật_liệu điện_sắc được giới khoa_học...,SUPPORTED,khoahoc
77049,0,10830,những chiến_thắng được ca_ngợi giúp prigozhin ...,các thành_viên wagner được cho là đã âm_thầm t...,điều đó cũng giúp prigozhin tăng_cường ảnh_hưở...,SUPPORTED,thegioi
198296,0,19911,những bức ảnh được nhiếp_ảnh gia_john thomson ...,áo_dài năm thân_may theo phong_cách kín_đáo đư...,thiếu_nữ sài_gòn là một trong những bức ảnh đư...,SUPPORTED,thegioi


In [52]:
df_eval_1 = df[df['evidence_label'] == 1].head(690)
df_eval_0 = df[df['evidence_label'] == 0].head(690)
df_eval = pd.concat([df_eval_0, df_eval_1])

In [53]:
df_train = df[~df.isin(df_eval).all(axis=1)]
df_train_1 = df_train[df_train['evidence_label'] == 1]
df_train_0 = df_train[df_train['evidence_label'] == 0].head(len(df_train_1)*args.scale_pos_neg)
df_train = pd.concat([df_train_0, df_train_1])

In [54]:
# df_train_1 = df[df['evidence_label'] == 1]
# df_train_0 = df[df['evidence_label'] == 0].head(len(df_train_1)*args.scale_pos_neg)
# df_train = pd.concat([df_train_0, df_train_1])

In [55]:
df_train['evidence_label'].value_counts()


0    132560
1     13256
Name: evidence_label, dtype: int64

In [56]:
df_eval['evidence_label'].value_counts()


0    690
1    690
Name: evidence_label, dtype: int64

In [70]:
# debug
df_eval = df_eval.head(10)
df_train = df_train.head(100)

# Model

## Colate funtion

In [71]:
def collate_to_max_length(batch: List[List[torch.Tensor]], max_len: int = None, fill_values: List[float] = None) -> \
    List[torch.Tensor]:
    """
    pad to maximum length of this batch
    Args:
        batch: a batch of samples, each contains a list of field data(Tensor), which shape is [seq_length]
        max_len: specify max length
        fill_values: specify filled values of each field
    Returns:
        output: list of field batched data, which shape is [batch, max_length]
    """
    # [batch, num_fields]
    lengths = np.array([[len(field_data) for field_data in sample] for sample in batch])
    batch_size, num_fields = lengths.shape
    fill_values = fill_values or [0.0] * num_fields
    # [num_fields]
    max_lengths = lengths.max(axis=0)
    if max_len:
        assert max_lengths.max() <= max_len
        max_lengths = np.ones_like(max_lengths) * max_len

    output = [torch.full([batch_size, max_lengths[field_idx]],
                         fill_value=fill_values[field_idx],
                         dtype=batch[0][field_idx].dtype)
              for field_idx in range(num_fields)]
    for sample_idx in range(batch_size):
        for field_idx in range(num_fields):
            # seq_length
            data = batch[sample_idx][field_idx]
            output[field_idx][sample_idx][: data.shape[0]] = data
    # generate span_index and span_mask
    max_sentence_length = max_lengths[0]
    start_indexs = []
    end_indexs = []
    for i in range(1, max_sentence_length - 1):
        for j in range(i, max_sentence_length - 1):
            # # span大小为10
            # if j - i > 10:
            #     continue
            start_indexs.append(i)
            end_indexs.append(j)
    # generate span mask
    span_masks = []
    for input_ids, label, length in batch:
        span_mask = []
        middle_index = input_ids.tolist().index(2)
        for start_index, end_index in zip(start_indexs, end_indexs):
            if 1 <= start_index <= length.item() - 2 and 1 <= end_index <= length.item() - 2 and (
                start_index > middle_index or end_index < middle_index):
                span_mask.append(0)
            else:
                span_mask.append(1e6)
        span_masks.append(span_mask)
    # add to output
    output.append(torch.LongTensor(start_indexs))
    output.append(torch.LongTensor(end_indexs))
    output.append(torch.LongTensor(span_masks))
    return output  # (input_ids, labels, length, start_indexs, end_indexs, span_masks)



## Data loader

In [72]:
import json
import os
from functools import partial

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer

class DSC_Dataset(Dataset):
    def __init__(self, dataframe, bert_path, max_length: int = 256):
        super().__init__()
        self.max_length = max_length
        # label_map = {"NON": 0, 'SR': 1}
        # with open(os.path.join(directory, prefix + '.json')) as file:
        #     data = json.load(file)
        # self.result = data
        self.result = dataframe
        self.tokenizer = AutoTokenizer.from_pretrained(bert_path, use_fast=False)
        # self.tokenizer = RobertaTokenizer.from_pretrained(bert_path)

    def __len__(self):
        return len(self.result)

    def __getitem__(self, idx):
        # sentence_1, sentence_2, label = self.result[idx]
        # check for required keys
        # if not all(key in self.result[idx] for key in ["claim", "sentence", "verdict"]):
        #     raise KeyError("Missing key in result dictionary.")

        sentence_1 = self.result.iloc[idx]["claim"]
        sentence_2 = self.result.iloc[idx]["sentence"]
        label = self.result.iloc[idx]["evidence_label"]
        # remove .
        if sentence_1.endswith("."):
            sentence_1 = sentence_1[:-1]
        if sentence_2.endswith("."):
            sentence_2 = sentence_2[:-1]
        sentence_1_input_ids = self.tokenizer.encode(
            sentence_1, add_special_tokens=False)
        sentence_2_input_ids = self.tokenizer.encode(
            sentence_2, add_special_tokens=False)
        input_ids = sentence_1_input_ids + [2] + sentence_2_input_ids
        if len(input_ids) > self.max_length - 2:
            input_ids = input_ids[:self.max_length - 2]
        # convert list to tensor
        length = torch.LongTensor([len(input_ids) + 2])
        input_ids = torch.LongTensor([0] + input_ids + [2])
        label = torch.LongTensor([label])
        return input_ids, label, length


# def unit_test():
#     root_path = "D:\\DUCBUI\\DSC\\self_explanining\\data\\"
#     bert_path = "roberta-base"
#     prefix = "train_evidence"
#     dataset = DSC_Dataset(directory=root_path,
#                           prefix=prefix, bert_path=bert_path)

#     print(dataset)



# if __name__ == '__main__':
#     unit_test()

In [73]:
def get_dataloader(args, dataset) -> DataLoader:
        """get training dataloader"""
        dataloader = DataLoader(
            dataset=dataset,
            batch_size=args.batch_size,
            num_workers=args.workers,
            collate_fn=partial(collate_to_max_length, fill_values=[1, 0, 0]),
            drop_last=False,
        )
        return dataloader

## Build model

In [74]:
class ExplainableModel(nn.Module):
    def __init__(self, bert_dir):
        super().__init__()
        self.bert_config = RobertaConfig.from_pretrained(
            bert_dir, output_hidden_states=True)
        self.intermediate = RobertaModel.from_pretrained(bert_dir)
        self.span_info_collect = SICModel(self.bert_config.hidden_size)
        self.interpretation = InterpretationModel(self.bert_config.hidden_size)
        self.output = nn.Linear(
            self.bert_config.hidden_size, self.bert_config.num_labels)

    def forward(self, input_ids, start_indexs, end_indexs, span_masks):
        # generate mask
        attention_mask = (input_ids != 1).long()
        # intermediate layer
        a = self.intermediate(
            input_ids, attention_mask=attention_mask)
        hidden_states = self.intermediate(
            input_ids, attention_mask=attention_mask).last_hidden_state
        # span info collecting layer(SIC)
        h_ij = self.span_info_collect(hidden_states, start_indexs, end_indexs)
        # interpretation layer
        H, a_ij = self.interpretation(h_ij, span_masks)
        # output layer
        out = self.output(H)
        return out, a_ij


class SICModel(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size

        self.W_1 = nn.Linear(hidden_size, hidden_size)
        self.W_2 = nn.Linear(hidden_size, hidden_size)
        self.W_3 = nn.Linear(hidden_size, hidden_size)
        self.W_4 = nn.Linear(hidden_size, hidden_size)

    def forward(self, hidden_states, start_indexs, end_indexs):
        W1_h = self.W_1(hidden_states)  # (bs, length, hidden_size)
        W2_h = self.W_2(hidden_states)
        W3_h = self.W_3(hidden_states)
        W4_h = self.W_4(hidden_states)

        # (bs, span_num, hidden_size)
        W1_hi_emb = torch.index_select(W1_h, 1, start_indexs)
        W2_hj_emb = torch.index_select(W2_h, 1, end_indexs)
        W3_hi_start_emb = torch.index_select(W3_h, 1, start_indexs)
        W3_hi_end_emb = torch.index_select(W3_h, 1, end_indexs)
        W4_hj_start_emb = torch.index_select(W4_h, 1, start_indexs)
        W4_hj_end_emb = torch.index_select(W4_h, 1, end_indexs)

        # [w1*hi, w2*hj, w3(hi-hj), w4(hi⊗hj)]
        span = W1_hi_emb + W2_hj_emb + \
            (W3_hi_start_emb - W3_hi_end_emb) + \
            torch.mul(W4_hj_start_emb, W4_hj_end_emb)
        h_ij = torch.tanh(span)
        return h_ij


class InterpretationModel(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.h_t = nn.Linear(hidden_size, 1)

    def forward(self, h_ij, span_masks):
        o_ij = self.h_t(h_ij).squeeze(-1)  # (ba, span_num)
        # mask illegal span
        o_ij = o_ij - span_masks
        # normalize all a_ij, a_ij sum = 1
        a_ij = nn.functional.softmax(o_ij, dim=1)
        # weight average span representation to get H
        H = (a_ij.unsqueeze(-1) * h_ij).sum(dim=1)  # (bs, hidden_size)
        return H, a_ij


# def main():
#     # data
#     input_id_1 = torch.LongTensor([0, 4, 5, 6, 7, 2])
#     input_id_2 = torch.LongTensor([0, 4, 5, 2])
#     input_id_3 = torch.LongTensor([0, 4, 2])
#     batch = [(input_id_1, torch.LongTensor([1]), torch.LongTensor([6])),
#              (input_id_2, torch.LongTensor([1]), torch.LongTensor([4])),
#              (input_id_3, torch.LongTensor([1]), torch.LongTensor([3]))]

#     output = collate_to_max_length(batch=batch, fill_values=[1, 0, 0])
#     input_ids, labels, length, start_indexs, end_indexs, span_masks = output

#     # model
#     bert_path = "roberta-base"
#     model = ExplainableModel(bert_path)
#     print(model)

#     output = model(input_ids, start_indexs, end_indexs, span_masks)
#     print(output[0])


# if __name__ == '__main__':
#     main()


In [75]:
model = ExplainableModel(args.bert_path)
model

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ExplainableModel(
  (intermediate): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm

In [76]:
# Xuất các layer của mô hình
for name, module in model.named_modules():
    print(name, end="*******\n")
    print(module)
    # print(name)

*******
ExplainableModel(
  (intermediate): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (L

In [77]:
# Đếm số lượng tham số huấn luyện được
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Số lượng tham số huấn luyện của mô hình Transformer:", trainable_params)

Số lượng tham số huấn luyện của mô hình Transformer: 137362947


In [78]:

# Đóng băng các lớp trước layer11.output
layer11_found = False
for name, param in model.named_parameters():
    if name.startswith("intermediate.encoder.layer.11.output"):
        layer11_found = True
    if not layer11_found:
        param.requires_grad_(False)
        param.grad = None

In [79]:
# Đếm số lượng tham số huấn luyện được
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Số lượng tham số huấn luyện của mô hình Transformer:", trainable_params)

Số lượng tham số huấn luyện của mô hình Transformer: 5316867


# Train

In [80]:

# METRICS EVAL
def acc_and_f1(preds, labels):
    acc = accuracy_score(y_true=labels, y_pred=preds)
    f1 = f1_score(y_true=labels, y_pred=preds, average="macro")
    return {
        "acc": acc,
        "f1": f1,
        "acc_and_f1": (acc + f1) / 2,
    }

def pearson_and_spearman(preds, labels):
    pearson_corr = pearsonr(preds, labels)[0]
    spearman_corr = spearmanr(preds, labels)[0]
    return {
        "pearson": pearson_corr,
        "spearmanr": spearman_corr,
        "corr": (pearson_corr + spearman_corr) / 2,
    }


In [81]:
model = ExplainableModel(args.bert_path)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(args.bert_path, use_fast=False)
model

# for name, param in model.named_parameters():
#     param.requires_grad_(True)

layer11_found = False
for name, param in model.named_parameters():
    if name.startswith("intermediate.encoder.layer.11"):
        layer11_found = True
    if not layer11_found:
        param.requires_grad_(False)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Số lượng tham số huấn luyện của mô hình Transformer:", trainable_params)


Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Số lượng tham số huấn luyện của mô hình Transformer: 10043139


In [82]:
# Prepare optimizer
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters,
                  betas=(0.9, 0.98),  # according to RoBERTa paper
                  lr=args.lr,
                  eps=args.adam_epsilon)

global_step = 0
nb_tr_steps = 0
tr_loss = 0
num_labels = args.num_labels

logger.info(f"Optimzer:{optimizer.__class__.__name__}")


data_train = DSC_Dataset(df_train,
                      bert_path=args.bert_path,
                      max_length=args.max_length)

data_dev = DSC_Dataset(df_eval,
                      bert_path=args.bert_path,
                      max_length=args.max_length)

num_train_optimization_steps = int(
            len(data_train) / args.batch_size / args.gradient_accumulation_steps) * args.epochs

kfold = KFold(n_splits=args.kfold, shuffle=True, random_state=42)

logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(data_train))
logger.info("  Batch size = %d", args.batch_size)
logger.info("  Num steps = %d", num_train_optimization_steps)

# data_fold = {"train": [], "test": []}
# for fold_train, fold_test in kfold.split(data_train):
#     data_fold["train"].append(fold_train)
#     data_fold["test"].append(fold_test)

best_acc_and_f1 = 0
for epoch in trange(int(args.epochs), desc="Epoch"):
  model.train()
  label_pred = []
  label_true = []
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0

  # idx_train = data_fold["train"][epoch]
  # idx_eval = data_fold["test"][epoch]

  # train_sampler = SubsetRandomSampler(idx_train)

  train_dataloader = get_dataloader(args=args, dataset=data_train)


  # eval_data = Subset(data_train,idx_eval)
  # eval_sampler = RandomSampler(eval_data)
  eval_dataloader = get_dataloader(args=args, dataset=data_dev)

  for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
    batch = tuple(t.to(device) for t in batch)
    input_ids, label, length, start_index, end_index, span_mask = batch
    input_ids = input_ids.to(device)
    label = label.to(device)
    start_index = start_index.to(device)
    end_index = end_index.to(device)
    span_mask = span_mask.to(device)


    with torch.no_grad():
        output_logits = model(input_ids=input_ids,
                                start_indexs=start_index,
                                end_indexs=end_index,
                                span_masks=span_mask)

        loss_fct = CrossEntropyLoss(weight=torch.tensor([1., args.scale_pos_neg]).to(device))
        logits = output_logits[0].detach()
        label = label.view(-1).detach()
        loss = loss_fct(logits, label)

        # Compute loss, evaluation
        probs = torch.nn.functional.softmax(logits, dim=-1)
        _, pred = torch.max(probs, dim=-1)

        label_pred.extend(pred.cpu().numpy())
        label_true.extend(label.cpu().numpy())

        loss.requires_grad = True


        if n_gpu > 1:
            loss = loss.mean()  # mean() to average on multi-gpu.
        if args.gradient_accumulation_steps > 1:
            loss = loss / args.gradient_accumulation_steps

        if args.fp16:
            optimizer.backward(loss)
        else:
            loss.backward()

        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1

        if (step + 1) % args.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
  print('printing loss')
  print('training_loss~=',tr_loss/nb_tr_steps)
  acc_f1 = acc_and_f1(preds=label_pred, labels=label_true)
  print(acc_f1)
  if acc_f1['acc_and_f1'] > best_acc_and_f1:
    best_acc_and_f1 = acc_f1["acc_and_f1"]
    if args.do_train and (args.local_rank == -1 or torch.distributed.get_rank() == 0):
        # Save a trained model, configuration and tokenizer
        model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

        # If we save using the predefined names, we can load using `from_pretrained`
        output_model_file = os.path.join(args.save_path, f"{WEIGHTS_NAME}_{epoch}")
        output_config_file = os.path.join(args.save_path, CONFIG_NAME)

        torch.save(model_to_save.state_dict(), output_model_file)
        if epoch == 0:
            tokenizer.save_vocabulary(args.save_path)


  print("========= Evaluating ========= ")

  model.eval()
  eval_loss = 0
  nb_eval_steps = 0
  preds = []
  preds = []
  labels = []
  probs = []
  store_output=list()
  # softmaxing=torch.nn.Softmax()
  for input_ids, label, length, start_index, end_index, span_mask in tqdm(eval_dataloader, desc="Evaluating"):
    input_ids = input_ids.to(device)
    label = label.to(device)
    start_index = start_index.to(device)
    end_index = end_index.to(device)
    span_mask = span_mask.to(device)
    #
    model.zero_grad()
    with torch.no_grad():

        output_logits = model(input_ids=input_ids,
                            start_indexs=start_index,
                            end_indexs=end_index,
                            span_masks=span_mask)
        # logits=softmaxing(logits)
        logits = output_logits[0]
        label = label.view(-1)

        store_output.extend(logits.cpu().numpy())
        prob_in_batch = torch.nn.functional.softmax(logits, dim=-1)

        # Chọn nhãn với xác suất cao nhất
        prob, pred = torch.max(prob_in_batch, dim=-1)
        preds.extend(pred.cpu().numpy())
        labels.extend(label.cpu().numpy())
        probs.extend(prob.cpu().numpy())
  print(acc_and_f1(preds=preds, labels=labels))

#   if args.do_train and (args.local_rank == -1 or torch.distributed.get_rank() == 0):
#       # Save a trained model, configuration and tokenizer
#       model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

#       # If we save using the predefined names, we can load using `from_pretrained`
#       output_model_file = os.path.join(args.save_path, f"{WEIGHTS_NAME}_{epoch}")
#       output_config_file = os.path.join(args.save_path, CONFIG_NAME)

#       torch.save(model_to_save.state_dict(), output_model_file)
#       if epoch == 0:
#           tokenizer.save_vocabulary(args.save_path)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

printing loss
training_loss~= 0.6616931676864624
{'acc': 0.93, 'f1': 0.48186528497409326, 'acc_and_f1': 0.7059326424870467}
========= Evaluating ========= 


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch:  10%|█         | 1/10 [00:07<01:08,  7.61s/it]

...

Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

printing loss
training_loss~= 0.6614616453647614
{'acc': 0.92, 'f1': 0.4791666666666667, 'acc_and_f1': 0.6995833333333333}
========= Evaluating ========= 


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 100%|██████████| 10/10 [00:27<00:00,  2.72s/it]

{'acc': 0.7, 'f1': 0.4117647058823529, 'acc_and_f1': 0.5558823529411765}


# Test

In [83]:
#  if args.do_test and (args.local_rank == -1 or torch.distributed.get_rank() == 0):
#   # model.load_state_dict(torch.load(os.path.join(args.output_dir,f"pytorch_model_{epoch}.bin")))
#     test_examples = processor.get_test_examples(args.data_dir)
#     # test_examples=test_examples[0:500] #debugging
#     test_features = convert_examples_to_features_test(
#         test_examples, label_list, args.max_seq_length, tokenizer, output_mode)
#     logger.info("***** Running testing *****")
#     logger.info("  Num examples = %d", len(test_examples))
#     logger.info("  Batch size = %d", args.eval_batch_size)
#     all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
#     all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
#     all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)

#     test_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)
#     # Run prediction for full data
#     test_sampler = SequentialSampler(test_data)
#     test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=args.eval_batch_size)

#     model.eval()
#     preds = []
#     probs = []
#     store_output=list()
#     # softmaxing=torch.nn.Softmax()
#     for input_ids, input_mask, segment_ids in tqdm(test_dataloader, desc="Evaluating"):
#         input_ids = input_ids.to(device)
#         input_mask = input_mask.to(device)
#         segment_ids = segment_ids.to(device)

#         with torch.no_grad():
#             output_logits = model(input_ids=input_ids, token_type_ids=None, attention_mask=input_mask, labels=None)
#             store_output.extend(output_logits.logits.cpu().numpy())
#             prob_in_batch = torch.nn.functional.softmax(output_logits.logits, dim=-1)

#             # Chọn nhãn với xác suất cao nhất
#             prob, pred = torch.max(prob_in_batch, dim=-1)
#             preds.extend(pred.cpu().numpy())
#             probs.extend(prob.cpu().numpy())
#     # torch.from_numpy(np.array(store_output))
#     print("Storing dev scores")
#     claim_ids = []
#     num_sentences = []
#     claims =[]
#     evidences = []
#     ids = []
#     for example in test_examples:
#         id, claim_id, num_sentence, claim, evidence = processor.get_info_eval(example)
#         ids.append(id)
#         claim_ids.append(claim_id)
#         num_sentences.append(num_sentence)
#         claims.append(claim)
#         evidences.append(evidence)

#     data = {
#         "claim_id": claim_ids,
#         "num_sentence": num_sentences,
#         "claim": claims,
#         "evidence": evidences,
#         "pred": preds,
#         "prob": probs
#     }
#     eval_df = pd.DataFrame(data=data)
#     eval_df.to_json(os.path.join(args.save_path,"test_rationale.json"), force_ascii=False, indent=4, orient='index')
